In [25]:
# import Libraries

import openai
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os

In [7]:
## Lets Read the document
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [15]:
doc=read_doc('documents/')
len(doc)

63

In [16]:
doc

[Document(page_content='2/6/24, 7:18 PM 사람인  인성검사  - 사람인\nhttps://www.saramin.co.kr/zf_user/member/svq/exam-result?apply_id=4097369&job_code=13 1/5\n사람인 인성검사 결과응시자\n신승엽희망직무\nIT·인터넷검사일자\n2024.01.21\n표준점수 인성검사 응시자 집단에서 나의 상대적인 위치를 나타내기 위해 산출하는 점수\n백분위 나보다 낮은 점수를 받은 사람을 백분율로 표기\n예) 점수가 95%라면 내 점수보다 낮은 사람이 95% 있다는 것을 의미\n응답신뢰도 일관적인 답변을 하지 않았거나 실제 자신의 성향보다 더 긍정적인 방향으로 응답하면 신뢰도가 낮아질 수 있음직무적합(IT·인터넷) : 높음\nIT·인터넷직에서는 작은 것도 놓치지 않는 세밀함, 논리적이고 체계적인 사고력 및 업무적으로 타인에게 신뢰감을 심어줄 수\n있는 자세, 주도적인 태도로 새롭거나 대안적인 방안을 제시하려는 자세가 요구됩니다.\n신승엽님은 해당 직무에 대한 적합도가 매우 높아 주어진 직무 상황에 잘 적응하며, 우수한 역량을 발휘할 가능성이 높습니다.* 본 검사는 응답을 토대로 작성되었으며, 본인의 직무 적합도를 확인할 수 있습니다. 종합 결과\n표준점수\n69 점백분위\n93 %응답신뢰도\n낮음\n인성 요인 비교\n정직겸손성\n정서안정성\n외향성\n원만성성실성개방성내 점수 평균 점수\n요인별 평가', metadata={'source': 'documents/sample.pdf', 'page': 0}),
 Document(page_content='2/6/24, 7:18 PM 사람인  인성검사  - 사람인\nhttps://www.saramin.co.kr/zf_user/member/svq/exam-result?apply_id=4097369&job_code=13 2/5정직겸손성 자신에게 유리한 방향으로 의사 결정하거나 사람들을 조정하지 않고, 물질적인 것에 과하게 욕심을 내지 않으

In [17]:
## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc

In [18]:
documents=chunk_data(docs=doc)
documents

[Document(page_content='2/6/24, 7:18 PM 사람인  인성검사  - 사람인\nhttps://www.saramin.co.kr/zf_user/member/svq/exam-result?apply_id=4097369&job_code=13 1/5\n사람인 인성검사 결과응시자\n신승엽희망직무\nIT·인터넷검사일자\n2024.01.21\n표준점수 인성검사 응시자 집단에서 나의 상대적인 위치를 나타내기 위해 산출하는 점수\n백분위 나보다 낮은 점수를 받은 사람을 백분율로 표기\n예) 점수가 95%라면 내 점수보다 낮은 사람이 95% 있다는 것을 의미\n응답신뢰도 일관적인 답변을 하지 않았거나 실제 자신의 성향보다 더 긍정적인 방향으로 응답하면 신뢰도가 낮아질 수 있음직무적합(IT·인터넷) : 높음\nIT·인터넷직에서는 작은 것도 놓치지 않는 세밀함, 논리적이고 체계적인 사고력 및 업무적으로 타인에게 신뢰감을 심어줄 수\n있는 자세, 주도적인 태도로 새롭거나 대안적인 방안을 제시하려는 자세가 요구됩니다.\n신승엽님은 해당 직무에 대한 적합도가 매우 높아 주어진 직무 상황에 잘 적응하며, 우수한 역량을 발휘할 가능성이 높습니다.* 본 검사는 응답을 토대로 작성되었으며, 본인의 직무 적합도를 확인할 수 있습니다. 종합 결과\n표준점수\n69 점백분위\n93 %응답신뢰도\n낮음\n인성 요인 비교\n정직겸손성\n정서안정성\n외향성\n원만성성실성개방성내 점수 평균 점수\n요인별 평가', metadata={'source': 'documents/sample.pdf', 'page': 0}),
 Document(page_content='2/6/24, 7:18 PM 사람인  인성검사  - 사람인\nhttps://www.saramin.co.kr/zf_user/member/svq/exam-result?apply_id=4097369&job_code=13 2/5정직겸손성 자신에게 유리한 방향으로 의사 결정하거나 사람들을 조정하지 않고, 물질적인 것에 과하게 욕심을 내지 않으

In [31]:
## Embedding Technique Of OPENAI
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])

In [27]:
vectors=embeddings.embed_query("How are you?")
len(vectors)

1536

In [28]:
vectors

[-0.016774665377855796,
 -0.012071833811637811,
 0.0066777732412390545,
 -0.025995175996580647,
 -0.016157494678324034,
 0.017614014250963563,
 -0.011139907750351919,
 -0.00989939727062271,
 -0.018169468066806662,
 -0.010442506405876486,
 0.027797311235463768,
 0.0016432135416098253,
 -0.0072825992601814925,
 -0.011695360634872453,
 0.007227053785464926,
 -0.015392205506849124,
 0.028365106937928496,
 -0.011917541602416154,
 0.01405911970158692,
 -0.020650489026265078,
 0.0024393622026669527,
 0.006375360231767835,
 0.0010052146278355383,
 -0.008220696730439481,
 -0.01592297089283614,
 -0.0077146179116634175,
 0.02505707899198394,
 -0.01244213511376979,
 0.022304502999237356,
 -0.025143481511560994,
 0.005637842633498002,
 0.007733132604240991,
 -0.013182738649356313,
 0.004014687102370056,
 0.008807007125481782,
 -0.022304502999237356,
 0.004042459606897699,
 -0.010491880471620956,
 0.020341904607821763,
 -0.006381531640739933,
 0.02705670676855468,
 0.0012759977111332535,
 -0.0052644

In [14]:
## Vector Search DB In Pinecone
pinecone.init(
    api_key="923d5299-ab4c-4407-bfe6-7f439d9a9cb9",
    environment="gcp-starter"
)
index_name="langchainvector"

In [15]:
index=Pinecone.from_documents(doc,embeddings,index_name=index_name)

In [17]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [18]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [19]:
llm=OpenAI(model_name="text-davinci-003",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

In [22]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [24]:
our_query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content="7 \n \n \n farmers in contributing to the health of fellow citizens by growing these \n‘Shree Anna’.   \n22. Now to make India a global hub for ' Shree Anna' , the Indian Institute \nof Millet Research, Hyderabad  will be supported as the Centre of Excellence \nfor sharing best practices, research and technologies at the international \nlevel.    \nAgriculture Credit  \n23. The agriculture credit target will be increased  \nto ` 20 lakh crore with focus on animal husbandry, dairy and fisheries.  \nFisheries \n24. We will launch a new sub-scheme of PM Matsya Sampada Yojana \nwith targeted investment of ` 6,000 crore to further enable activities of \nfishermen, fish vendors, and micro & small enterprises, improve value chain \nefficiencies, and expand the market. \nCooperation \n25. For farmers, especially small and marginal farmers, and other \nmarginalised sections, the government is promoting cooperative-based \neconomic development model. A new Ministry of Coo